#  training_message.py 改修前の動作確認

## (1) DBテストデータの仕込み

### (1-1) 環境の準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    該当Botのファイルパスを用意
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
    'test_daikin_conversation.csv',
    'test_ptna_conversation.csv',
    'test_toyotsu_human_conversation.csv',
]

learning_file_paths = []
for csv_file_name in csv_file_names:
    learning_csv_dir = os.path.join(learning_dir, 'learning/tests/fixtures/')
    learning_file_path = os.path.join(learning_csv_dir, csv_file_name)
    learning_file_paths.append(learning_file_path)

learning_file_paths

['/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/fixtures/test_benefitone_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/fixtures/test_daikin_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/fixtures/test_ptna_conversation.csv',
 '/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/fixtures/test_toyotsu_human_conversation.csv']

### (1-2) データ操作用関数の準備

In [3]:
import numpy as np
import pandas as pd

import MySQLdb
db_config = {
    'host': 'localhost',
    'db': 'donusagi_bot',
    'user': 'root',
    'passwd': '',
    'charset': 'utf8'
}

In [4]:
def insert_record_from_csv(bot_id, learning_file_path):
    print('insert_record_from_csv start [%s]' % os.path.basename(learning_file_path))
    
    '''
        CSVファイルからデータフレームを生成
    '''
    dataframe = pd.read_csv(learning_file_path, encoding='utf-8')
    ids = dataframe['id']
    questions = dataframe['question']
    answer_ids = dataframe['answer_id']
    answer_bodys = dataframe['answer_body']

    '''
        INSERT文を準備
    '''
    sql_statement = "INSERT INTO learning_training_messages (bot_id, id, question, answer_id, answer_body, created_at, updated_at) VALUES (%s,%s,%s,%s,%s,%s,%s)"

    '''
        データフレームをDBに投入
    '''
    db = MySQLdb.connect(**db_config)
    try:
        cursor = db.cursor()
        for index, _ in enumerate(ids):
            data = (
                bot_id, 
                ids[index], 
                questions[index], 
                answer_ids[index], 
                answer_bodys[index],
                '2017-05-02 10:00:00',
                '2017-05-02 10:00:00'
            )
            cursor.execute(sql_statement, data)

    except Exception as e:
        print(e)
        db.rollback()
        db.close()
        return

    '''
        DB変更内容の反映
    '''
    db.commit()
    db.close()
    print('insert_record_from_csv end')

In [5]:
def delete_record_from_table(bot_id):
    '''
        DELETE文を準備
    '''
    sql_statement = "DELETE FROM learning_training_messages where bot_id=%s"

    '''
        DBから指定Botのレコードを削除
    '''
    db = MySQLdb.connect(**db_config)
    try:
        cursor = db.cursor()
        cursor.execute(sql_statement % bot_id)

    except Exception as e:
        print(e)
        db.rollback()
        db.close()
        return

    '''
        DB変更内容の反映
    '''
    db.commit()
    db.close()

### (1-3) データ投入

In [6]:
'''
    テスト用Botの全レコードを投入
'''
for i, learning_file_path in enumerate(learning_file_paths):
    bot_id = 10001 + i
    insert_record_from_csv(bot_id, learning_file_path)

    # こちらは削除用
    # delete_record_from_table(bot_id)

insert_record_from_csv start [test_benefitone_conversation.csv]
insert_record_from_csv end
insert_record_from_csv start [test_daikin_conversation.csv]
insert_record_from_csv end
insert_record_from_csv start [test_ptna_conversation.csv]
insert_record_from_csv end
insert_record_from_csv start [test_toyotsu_human_conversation.csv]
insert_record_from_csv end


## (2) 動作確認

まずは bin/learn.py により、単独Bot [ptna] のデータで動作確認。

ローカル環境に問題がないことを確認できました。

```
MacBookPro-makmorit-jp:learning makmorit$ python3 -m bin.learn --bot_id=10001
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
2017/05/02 AM 11:52:52 learning_parameter: {'_vectorize_using_all_bots': True, '_include_tag_vector': False, '_params_for_algorithm': {}, '_use_similarity_classification': False, '_include_failed_data': False, '_algorithm': 0, '_excluded_labels_for_fitting': [], '_classify_threshold': None}
2017/05/02 AM 11:52:52 start Bot#learn
{'database': {'name': 'donusagi_bot', 'endpoint': 'mysql://root@localhost/donusagi_bot?charset=utf8', 'password': '', 'host': 'localhost', 'user': 'root'}}
2017/05/02 AM 11:52:52 Bot after mysql connect
2017/05/02 AM 11:52:52 TrainingMessage#__init__ start
2017/05/02 AM 11:52:52 TrainingMessage#build start
2017/05/02 AM 11:52:52 data['id'].count(): 4114
2017/05/02 AM 11:52:52 TextArray#__init__ start
2017/05/02 AM 11:52:53 TextArray#to_vec start
2017/05/02 AM 11:52:54 TextArray#to_vec end
2017/05/02 AM 11:52:54 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [4677, 14], [4678, 2584], [4679, 14], [4680, 14], [4683, 14], [4686, 14], [4687, 8], [4690, 8], [4691, 14], [4692, 44], [4693, 30], [4700, 12], [4707, 18], [4708, 14], [4709, 14], [4710, 14], [4711, 14], [4712, 16], [4713, 14], [4718, 20], [4719, 8], [4720, 20], [4721, 8], [4724, 8], [4727, 14], [4728, 62], [4729, 26], [4730, 20], [4731, 14], [4732, 22], [4733, 14], [4734, 34], [4735, 12], [4738, 14], [4739, 14], [4740, 14], [4741, 8], [4742, 20], [4743, 14], [4744, 14], [4745, 20], [4750, 22], [4751, 20], [4752, 14], [4753, 8], [4754, 8], [4755, 14], [4756, 20], [4757, 8], [4758, 28], [4759, 14], [4760, 8], [4761, 20], [4762, 8], [4766, 14], [4767, 14], [4772, 60], [4776, 8], [4781, 14], [4782, 14], [4783, 20], [4786, 8], [4792, 20], [4793, 8], [4794, 8], [4795, 14], [4797, 42], [4798, 36], [4800, 14], [4802, 14], [4804, 14], [4807, 8], [4808, 106], [4817, 26], [4821, 24], [4824, 16], [4827, 8], [4830, 22], [4831, 14], [4833, 14], [4834, 14], [4835, 36], [4836, 30], [4837, 8]]
2017/05/02 AM 11:52:54 use algorithm: logistic regression
2017/05/02 AM 11:52:54 learning_parameter has not parameter C
2017/05/02 AM 11:52:57 best_params_: {'C': 140}
2017/05/02 AM 11:52:57 after Bot#__get_estimator
2017/05/02 AM 11:52:57 before Evaluator#evaluate
2017/05/02 AM 11:52:57 self.threshold: 0.0
0.981553398058
2017/05/02 AM 11:52:58 Evaluator#evaluate#elapsed time: 376.857996 ms
2017/05/02 AM 11:52:58 accuracy: 0.981553398058
2017/05/02 AM 11:52:58 end Bot#learn
2017/05/02 AM 11:52:58 Bot.learn processing time: 5.987392902374268 [sec]
2017/05/02 AM 11:52:58 MyopeServer.learn processing time: 5.987716913223267 [sec]
MacBookPro-makmorit-jp:learning makmorit$ 
```